In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

import dateparser as dp
import datetime

from PIL import Image
from io import BytesIO
import re

commons_site = pb.Site("commons", "commons")

In [ ]:
from modules.wmtools import is_commons_file, get_hash

In [ ]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [ ]:
req_proxy = RequestProxy()

In [ ]:
#### User input:
urls = ['http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=085c50bb1c']

#### User input:
categories = ['José María Aznar in 2001',
              'Historians from Spain']

In [ ]:
#### User input:
upload_categories = ['Files uploaded by User:Discasto']
categories = categories + upload_categories
categories

In [ ]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Fototeca: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

In [ ]:
used_names = []

for url in urls :
    cats = categories[:]
    while True:
        request = req_proxy.generate_proxied_request(url)
        if request is not None:
            #print("{0}".format(u''.join(request.text).encode('utf-8')))
            response = request.text
            break
        else :
            print ('Trying')

    soup = BeautifulSoup(response, 'html.parser')
    author = contents[0]
    if "Pool" not in author :
        print ("Image from a party other than Moncloa. Skipping")
        continue        

    title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
    try:
        date_string = soup.find_all("div", class_="contenidoFecha")[0].get_text()
        pub_date_sp = ' '.join(date_string.strip().split(' ')[-4:])
        parsed_pub_date = dp.parse(pub_date_sp)
        pub_date = parsed_pub_date.strftime('%Y-%m-%d')
    
        year_date = parsed_pub_date.strftime('%Y')
        if 'La Moncloa' in date_string:
            cats.append('Palacio de la Moncloa')
            cats.append(year_date + ' in Madrid')
    except :
        pub_date = None
    contents = soup.find_all("div", class_="entradillaContenedor")[0].get_text().strip().replace('\t', '').split('\r\n')
    contents = [item.replace('\n', '') for item in contents if len(item)>0]

    description = contents[1]
    image_url = 'http://www.lamoncloa.gob.es' + soup.find_all("a", id="ctl00_PlaceHolderMain_DisplayMode_aDescargar")[0]['href']

    vars = {
        "url": image_url,
        "description": description,
        "author": author,
        "date": pub_date,
        "title": title,
        "cat_string": '\n'.join(['[[Category:'+i+']]' for i in cats])
    }
    t = Template(template)
    _text = t.render(**vars)
    
    # First, the image is downloaded and stored
    image_name = '{}.jpg'.format(title.replace(':', ' -').replace('  ', ' '))
    image_path = os.path.join(images_directory, image_name)

    while True:
        print ('Trying download')
        img_request = req_proxy.generate_proxied_request(image_url)
        if img_request is not None:
            image = Image.open(BytesIO(img_request.content))
            image.save(image_path)
            print ('Image accessed')
            break
        else :
            print ('Still trying download')

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Leaving")
    else :
        # If the image name is already in commons, find a new name
        if pb.Page(commons_site, image_name, ns=6).exists():
            print ("Image name ({0}) already used in Commons".format(image_name))
            used_names.append(image_name)
        
        while True:
            if image_name in used_names :
                # Finding a new name
                image_subject = '.'.join(image_name.split('.')[:-1])
                image_extension = 'jpg'
                p = re.compile('(.*) ([0-9]{2}\.jpg)')
                m = p.match(image_name)
                if m is None:
                    image_name = image_subject + ' 01.' + image_extension
                else :
                    counter = int(m.group(2)[:2]) + 1
                    image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

                if pb.Page(commons_site, image_name, ns=6).exists():
                    print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                    used_names.append(image_name)
            else :
                print ("Preparing to upload image with name {0}".format(image_name))
                used_names.append(image_name)
                break

        # image upload
        bot = UploadRobot([image_path],
                          description = _text,
                          useFilename = image_name,
                          keepFilename = True,
                          verifyDescription = False,
                          ignoreWarning = True,
                          targetSite = commons_site)
        #bot.run()

    os.remove(image_path)